# Readmission Modeling: Script 1

In [1]:
import pandas as pd
import numpy as np
import os
import json
import pickle
from sklearn.model_selection import train_test_split

## Read original data and EDA

In [2]:
! aws s3 cp s3://cmsai-mrk-amzn/CSVModelInputs/readmission_input_targets_365_v2.csv data_d/

download: s3://cmsai-mrk-amzn/CSVModelInputs/readmission_input_targets_365_v2.csv to data_d/readmission_input_targets_365_v2.csv


In [3]:
# filepaths, feature names and target names
FP = './data/readmission_input_targets_365_v2.csv'
x_lst = [str(x) for x in range(365, -1, -1)]
y_lst = ['unplanned_readmission']

In [4]:
re_targets_365_df = pd.read_csv(FP)
re_targets_365_df.head()

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,5,6,7,8,9,12,15,16,17,18,19,20,21,22,23,24,25,26,28,29,30,32,33,34,36,37,38,40,43,50,52,53,54,55,57,60,68,70,71,73,74,75,76,77,78,81,82,88,90,91,92,93,94,95,97,98,99,102,103,104,106,107,108,109,110,112,116,118,119,120,121,124,125,126,129,133,134,135,136,137,138,139,140,144,145,146,149,154,155,158,159,162,163,165,166,167,168,172,173,176,177,179,181,183,184,186,187,188,189,190,192,194,195,196,200,201,202,203,205,208,209,215,216,217,218,220,222,223,225,227,228,230,232,246,247,248,251,253,254,256,258,261,273,277,293,302,303,310,314,325) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,patient_id,discharge_dt,discharge_id,365,364,363,362,361,360,359,...,8,7,6,5,4,3,2,1,0,unplanned_readmission
0,100002085,20110922,100002085_20110922,NaN,NaN,NaN,NaN,NaN,NaN,"d_7295, d_78650, d_78652, d_78659, d_78659, h_...",...,"d_486, d_5119, d_5128, d_5183, d_80709, h_7101...","d_496, d_80709, h_99232, h_99233","d_5119, d_80709, d_8600, h_00528, h_31645, h_3...","d_496, d_51889, h_71010","d_496, d_51189, d_5119, d_7931, d_80709, h_710...","d_5119, d_51919, d_7931, d_80709, h_00520, h_7...","d_5119, d_5128, d_5180, d_7931, d_80709, h_710...","d_5119, d_5183, d_80709, d_8600, d_V5399, h_32...","admission, d_496, d_72887, d_78605, d_78650, d...",False
1,100002829,20111013,100002829_20111013,NaN,"d_28521, d_58881, h_82310, h_84100",NaN,NaN,NaN,NaN,NaN,...,"h_90999, h_J1270",NaN,"h_90999, h_J1270",NaN,NaN,"h_90999, h_J1270, h_J1756","admission, d_40391, d_5856, d_5856, d_59970, d...","d_5856, d_59970, d_92303, d_9233, h_00400, h_1...","d_4019, d_5856, d_59970, discharge, h_90732, h...",False
2,100003379,20091207,100003379_20091207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"d_586, h_99231","d_99883, h_99231","d_586, d_99883, h_11042, h_99231, p_8622",NaN,"d_99883, h_99232",NaN,NaN,"admission, d_586, d_71945, d_V4989, discharge,...",True
3,100004211,20110102,100004211_20110102,NaN,NaN,NaN,"d_42731, d_42822, d_78650, h_93010, h_99214, h...","d_53081, d_78902, h_99214",NaN,NaN,...,"d_4019, d_42731, h_99233",NaN,"d_1950, d_5119, d_5738, h_71010, h_76705, h_99232","d_1539, d_V667, h_99233","d_1975, h_99233","d_1975, d_51881, h_99233","d_1975, d_42731, h_99233","d_1975, h_99233","d_1975, d_42731, d_51881, death, discharge, h_...",False
4,100008869,20101116,100008869_20101116,NaN,"d_29633, h_90806","d_53550, h_99213",NaN,NaN,NaN,NaN,...,"d_29620, h_99231","d_29620, h_99231",NaN,NaN,"admission, d_25000, d_29620, d_29623, d_4019, ...","d_41401, d_78650, h_93306, h_99222, h_99232","d_78650, h_99231",NaN,"d_78650, discharge, h_99238",False


In [5]:
# check duplicate IDs
re_targets_365_df.discharge_id.nunique()

1619157

In [6]:
re_targets_365_df.unplanned_readmission.value_counts(normalize=True)

False    0.855435
True     0.144565
Name: unplanned_readmission, dtype: float64

In [7]:
# remove duplicates
re_targets_365_df.drop_duplicates('discharge_id', inplace=True)
re_targets_365_df.shape

(1619157, 370)

In [8]:
re_targets_365_df.shape

(1619157, 370)

In [5]:
re_targets_365_df.shape

(1619158, 370)

**Remove death events**

In [9]:
np_re = re_targets_365_df.astype(str)
np_re['unplanned_readmission'] = re_targets_365_df[y_lst].astype(int).copy()
indeces = set()
for i in x_lst:
    indeces.update(np_re[np_re[i].str.contains('death')].index)
print(len(indeces)) #55568
np_re = np_re[~np_re.index.isin(indeces)]
print(np_re.shape) #(1563578, 370)

55569
(1563588, 370)


In [97]:
# previous runs
np_re = re_targets_365_df.astype(str)
np_re['unplanned_readmission'] = re_targets_365_df[y_lst].astype(int).copy()
indeces = set()
for i in x_lst:
    indeces.update(np_re[np_re[i].str.contains('death')].index)
print(len(indeces)) #55568
np_re = np_re[~np_re.index.isin(indeces)]
print(np_re.shape) #(1563578, 370)

55568
(1563578, 370)


**Save data with no more deaths**

In [12]:
import torch
torch.save(np_re, 'data_d/np_re')

**Check frequencies and length of sequences**

In [4]:
#reload if needed
np_re = torch.load('data_d/np_re')

In [17]:
re_targets_365_df.shape #(1619146, 370)

(1619157, 370)

In [49]:
# previous runs
re_targets_365_df.shape #(1619146, 370)

(1619146, 370)

In [21]:
# previous runs
test.size

86613670

In [13]:
words = np_re[x_lst[-30:]].values.ravel('K')

In [14]:
dic = {}
count = 0
for w in words:
    l = len(str(w).split(', '))
    if l not in dic:
        dic[l] = 1
    else:
        dic[l] += 1

In [15]:
for k, v in sorted(dic.items(), key=lambda x: x[1]):
    print(k, v)

81 1
94 1
89 1
110 1
80 1
90 1
157 1
99 1
111 1
77 1
91 1
76 2
121 2
74 2
86 2
79 2
105 2
97 2
85 2
103 2
98 3
73 4
78 4
75 6
70 7
69 8
71 8
63 9
72 9
65 10
67 10
66 11
68 12
64 14
61 22
60 24
62 24
58 36
59 40
56 44
57 49
55 79
54 84
53 97
52 124
50 139
51 143
49 164
48 196
47 226
46 300
45 330
44 400
43 458
42 548
41 718
40 804
39 966
38 1102
37 1341
36 1757
35 2055
34 2482
33 3109
32 3745
31 4466
30 5829
29 7091
28 9077
27 11777
26 14906
25 18858
24 24380
23 31080
22 39874
21 51007
20 65557
19 82750
18 103265
17 128467
16 157280
15 188086
14 222909
13 256422
12 311053
11 350046
10 432805
9 483982
8 630503
7 757443
6 1086147
5 1260608
4 1964510
3 1971199
2 3614046
1 32600467


In [16]:
len(words)

46907640

In [101]:
# previous runs
for k, v in sorted(dic.items(), key=lambda x: x[1]):
    print(k, v)

81 1
94 1
89 1
110 1
80 1
90 1
157 1
99 1
111 1
77 1
91 1
76 2
121 2
74 2
86 2
79 2
105 2
97 2
85 2
103 2
98 3
73 4
78 4
75 6
70 7
69 8
71 8
63 9
72 9
65 10
67 10
66 11
68 12
64 14
61 22
60 24
62 24
58 36
59 40
56 44
57 49
55 79
54 84
53 97
52 124
50 138
51 144
49 164
48 196
47 225
46 299
45 330
44 400
43 458
42 549
41 718
40 803
39 967
38 1101
37 1342
36 1757
35 2057
34 2479
33 3109
32 3747
31 4466
30 5832
29 7091
28 9081
27 11772
26 14914
25 18853
24 24402
23 31092
22 39875
21 51061
20 65635
19 82857
18 103430
17 128789
16 157707
15 188687
14 223796
13 256927
12 311452
11 349924
10 432975
9 483015
8 630505
7 756317
6 1086027
5 1259530
4 1964329
3 1971248
2 3613787
1 32600214


## Create vocabulary of words

In [18]:
import model_utils
from collections import Counter
import numbers
import time
import torch
import numbers
from collections import Counter
import torch
import torch.functional as F
from torch.nn.utils.rnn import pad_sequence
from torchtext.datasets import text_classification
from torchtext.vocab import Vocab

import metrics
import numpy as np

def build_vocab(df, feat_colnames, min_freq=1, specials=['<pad>', '<unk>'], specials_first=True):
    '''
    Create a vocabulary to be used with Script2. This maps all events to an index, including <pad> and <unk> and
    nan, which represents padding of sentences, unknown events, and no events respectively
    '''
    def build_counter(df, feat_colnames):
        counter = Counter()
        words = df[feat_colnames].values.ravel('K')
        print("start word number: ", words.shape)
        new_words = []
        for x in words:
            x = str(x)
            x = x.replace("d_s", "d_")
            new_words.extend(x.split(', '))
        print("exact word number: " , len(new_words))
        counter.update(new_words)
        if not isinstance(min_freq, numbers.Number):
            raise ValueError(f'Something wrong with {min_freq}')
        return counter
    

    pos_df = df[df['unplanned_readmission'] == True]
    pos_counter = build_counter(pos_df, feat_colnames)
    
    print(pos_counter['nan'])
    
    vocab = Vocab(pos_counter, min_freq=min_freq, specials=specials, specials_first=specials_first)

    print('Completed vocabulary')
    
    return vocab

In [19]:
pos_vocab = build_vocab(np_re, x_lst[-30:])
print(len(pos_vocab.stoi))

start word number:  (7021890,)
exact word number:  19767950
4363818
Completed vocabulary
22528


In [86]:
# previous runs
pos_vocab = build_vocab(np_re, x_lst[-30:])
print(len(pos_vocab.stoi))

start word number:  (7022430,)
exact word number:  19771134
4364148
Completed vocabulary
22529


'\nstart word number:  (7022430,)\nexact word number:  19771134\n4364148\nCompleted vocabulary\n29319\n'

In [20]:
# check 
pos_vocab['nan']

2

In [21]:
# save vocabulary for next script
torch.save(pos_vocab, 'data_d/pos_vocab_last30_non3digit')

## Train test validation splits with stratified targets with 365 dataset

Use 20% of data for testing, and the remaining (90%:10%) for training & validation

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    np_re[x_lst], np_re[y_lst],
    test_size=0.2,
    random_state=9999,
    stratify=np_re[y_lst])

In [6]:
print(X_train.shape)

(1250871, 366)


In [7]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train,
    test_size=0.1,
    random_state=9999,
    stratify=y_train)

In [8]:
print(X_train.shape, X_val.shape, X_test.shape)

(1125783, 366) (125088, 366) (312718, 366)


In [9]:
X_train.to_csv('./data/X_train.csv')
X_val.to_csv('./data/X_val.csv')
X_test.to_csv('./data/X_test.csv')

y_train.to_csv('./data/y_train.csv')
y_val.to_csv('./data/y_val.csv')
y_test.to_csv('./data/y_test.csv')

**Write to S3 for storage**

In [28]:
!aws s3 cp data/y_test.csv s3://cmsai-mrk-amzn/xianzeng/
!aws s3 cp data/X_test.csv s3://cmsai-mrk-amzn/xianzeng/
!aws s3 cp data/y_val.csv s3://cmsai-mrk-amzn/xianzeng/
!aws s3 cp data/X_val.csv s3://cmsai-mrk-amzn/xianzeng/
!aws s3 cp data/y_train.csv s3://cmsai-mrk-amzn/xianzeng/
!aws s3 cp data/X_train.csv s3://cmsai-mrk-amzn/xianzeng/

upload: data/y_test.csv to s3://cmsai-mrk-amzn/xianzeng/y_test.csv
upload: data/X_test.csv to s3://cmsai-mrk-amzn/xianzeng/X_test.csv  
upload: data/y_val.csv to s3://cmsai-mrk-amzn/xianzeng/y_val.csv 
upload: data/X_val.csv to s3://cmsai-mrk-amzn/xianzeng/X_val.csv    
upload: data/y_train.csv to s3://cmsai-mrk-amzn/xianzeng/y_train.csv
upload: data/X_train.csv to s3://cmsai-mrk-amzn/xianzeng/X_train.csv


## Train test validation splits with stratified targets with 30 day (max 30 events per day) dataset

Equivalent mostly to the 1000 events dataset.

In [23]:
np_re = torch.load('data/np_re_non3digit')

In [22]:
np_re = np_re[['patient_id', 'discharge_id'] + x_lst[-30:] + y_lst]

In [23]:
train_df, test_df = train_test_split(
    np_re,
    test_size=0.2,
    random_state=9999,
    stratify=np_re[y_lst])

In [24]:
train_df, val_df = train_test_split(
    train_df,
    test_size=0.1,
    random_state=9999,
    stratify=train_df[y_lst])

In [13]:
print(train_df.shape, val_df.shape, test_df.shape)

(1125783, 33) (125088, 33) (312718, 33)


In [15]:
# load vocabulary
import torch
pos_vocab = torch.load('data/pos_vocab_last30_non3digit')

In [17]:
# double check an event
pos_vocab.stoi['nan']

2

In [92]:
# quick check on vocabulary
count = 0
for key in pos_vocab.stoi.keys():
    count +=1
    print(key)
    if count > 30:
        break

<pad>
<unk>
nan
h_99232
h_99233
h_71010
admission
discharge
h_93010
h_99231
h_99223
h_A0425
d_4280
d_486
h_99285
d_42731
d_5856
h_71020
d_5849
h_99222
d_496
d_78650
d_78605
d_4019
d_51881
d_5990
h_99238
h_99213
h_99214
d_25000
h_36415


## Build dataset and save for script 2

In [25]:
import time
import random
def build_dataset(df, vocab, feat_colnames, label_colnames, day_length=30, max_length=50, mode='eval', keep_length=6):
    '''
    Subsets the entire dataset into a dataset to be used later.
    - Specific vocabulary
    - By number of days (whole dataset is 365)
    
    Returns a list of data and attributes needed by Script 2: patientid_dischargeid key, sequence of events, targets, and 
    mask (identifying padded regions)
    '''
    # create dataset
    start_time = time.time()
    print("used days: ", feat_colnames[-day_length], feat_colnames[-1])
    
    data = df[feat_colnames[-day_length:]].to_numpy()
    
    sequence = []
    count = 0
    pad_mask = []
    print("total size before: ", data.shape)
    
    # convert all events into indexes from vocab
    for i in range(len(data)):
        sentence = []
        mask = []
        
        for j in range(len(data[i])):
            words = str(data[i][j]).replace('d_s', 'd_').split(', ')
            words = sorted([vocab.stoi[w] if w in vocab.stoi else vocab.stoi['<unk>'] for w in words])
            
            if len(words) > max_length:
                words = words[:max_length]
            
            words = words + [vocab.stoi['<pad>']] * (max_length - len(words))
            sentence.append(words)
            mask.append([0.0 if w == vocab.stoi['<pad>'] else 1.0 for w in words])
            
        sequence.append(sentence)
        pad_mask.append(mask)
        
    finish_time = time.time()
    print("time: ", finish_time - start_time)
    print('New dataset created')
    print("random samples: ", count)
    print("sequence length: ", len(sequence))
    
    _, indices = np.unique(sequence, axis=0, return_index=True)
    indices = np.sort(indices)
    labels = df[label_colnames].to_numpy()[indices]
    discharge_ids = df['discharge_id'].to_numpy()[indices]
    pad_mask = np.array(pad_mask)[indices]
    
    try:
        sequence = np.array(sequence)[indices]
        print("unique sequence length: ", len(sequence))
        assert labels.shape[0] == sequence.shape[0]
    except:
        print("more processing for sequence")
        pass
    
    return [discharge_ids, sequence, labels, pad_mask]

In [21]:
#val_dataset = build_dataset(val_df, relative_pos_vocab, x_lst, y_lst, day_length=30, max_length=50)
val_dataset = build_dataset(val_df, pos_vocab, x_lst, y_lst, day_length=30, max_length=30)

used days:  29 0
total size before:  (125088, 30)
time:  42.632025718688965
New dataset created
random samples:  0
sequence length:  125088
unique sequence length:  125024


In [22]:
pickle.dump(val_dataset, open("data/val_dataset_last30_non3digit_latest.pkl", 'wb'), protocol=4)

In [23]:
#test_dataset = build_dataset(test_df, relative_pos_vocab, x_lst, y_lst, day_length=30, max_length=50)
test_dataset = build_dataset(test_df, pos_vocab, x_lst, y_lst, day_length=30, max_length=50)

used days:  29 0
total size before:  (312718, 30)
time:  138.9652864933014
New dataset created
random samples:  0
sequence length:  312718
unique sequence length:  312542


In [24]:
pickle.dump(test_dataset, open("data/test_dataset_last30_non3digit_latest.pkl", 'wb'), protocol=4)

In [25]:
train_dataset = None
#train_dataset = build_dataset(train_df, relative_pos_vocab, x_lst, y_lst, day_length=30, max_length=50, mode='train', keep_length=10)
train_dataset = build_dataset(train_df, pos_vocab, x_lst, y_lst, day_length=30, max_length=30)

used days:  29 0
total size before:  (1125783, 30)
time:  357.6096074581146
New dataset created
random samples:  0
sequence length:  1125783
unique sequence length:  1124923


In [26]:
pickle.dump(train_dataset, open("data/train_dataset_last30_non3digit_latest.pkl", 'wb'), protocol=4)

In [26]:
whole_dataset = build_dataset(np_re, pos_vocab, x_lst, y_lst, day_length=30, max_length=30)

used days:  29 0
total size before:  (1563588, 30)
time:  516.6843750476837
New dataset created
random samples:  0
sequence length:  1563588
unique sequence length:  1562234


In [94]:
# previous run
whole_dataset = build_dataset(np_re, pos_vocab, x_lst, y_lst, day_length=30, max_length=30)

used days:  29 0
total size before:  (1563578, 30)
time:  481.286744594574
New dataset created
random samples:  0
sequence length:  1563578
unique sequence length:  1562223


In [27]:
pickle.dump(whole_dataset, open("data_d/np_re_last30_non3digit_latest.pkl", 'wb'), protocol=4)

**Double check outputs manually to make sure mapping was correct between original and after indexing**

In [81]:
print(whole_dataset[3][0][:5])
print(whole_dataset[1][0][:5])

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]]
[[   2    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [  52   79  194  233  414  627  762  856 1092 1249 1490 7910    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [   2    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [  52   79  194    0    0    0    0    0    0    0  

In [73]:
np_re.iloc[0]

patient_id                                                       100000559
discharge_id                                            100000559_20111006
29                                                                     nan
28                       d_V5869, d_V5883, h_80186, h_82150, h_82247, h...
27                                                                     nan
26                                               h_90999, h_J1270, h_Q4081
25                                                                     nan
24                                                                     nan
23                             h_90999, h_G0008, h_J1270, h_Q2037, h_Q4081
22                                                                     nan
21                       d_V5861, h_85610, h_90999, h_J1270, h_J1756, h...
20                                                                     nan
19                                               h_90999, h_J1270, h_Q4081
18                       

In [68]:
pos_vocab.itos[79]

'h_Q4081'